Breadth-first search for finding stations within distance.
A_star for finding optimal route.
Use H&C as main axis due to lack of branching.

Nodes Completed
- Circle
    - Paddington to Hammersmith
    - Paddington to Bayswater
- Picadilly
    - Uxbridge to Cockfosters
    - Acton Town to Hatton Cross
    - Hatton Cross to Heathrow Terminals 2 & 3
    - Heathrow Terminal's 2 & 3 to Heathrow Terminal 5.

Matrix
- Bakerloo
- Circle
- District
- Jubilee
- Piccadilly

Pending
- Central
- Metropolitan
- Northern
- Victoria
- Waterloo and City

Not part of underground (in descending importance)
- Elizabeth
- London Trams
- Thameslink
- Liberty
- Lioness
- Mildmay
- Suffragette
- Weaver
- Windrush

Completed
- Circle
- District
- Picadilly
- H&C

In [1]:
from datetime import datetime
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Firefox()

# Initialise Google.
driver.get("https://www.google.com/maps")
driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Reject all"]').click()


In [ ]:
def get_undergound_data():
    
    driver.get("https://en.wikipedia.org/wiki/List_of_London_Underground_stations")
    stations: list = [station.text for station in driver.find_elements(By.CSS_SELECTOR, ".wikitable tbody tr th:first-child a")]
    areas: list = [area.text for area in driver.find_elements(By.CSS_SELECTOR, ".wikitable tbody tr td:last-child a")]

    lines = []
    for entry in driver.find_elements(By.CSS_SELECTOR, ".wikitable tbody tr td:nth-child(3)"):
        lines_str: str = re.sub(r"\[.\]","",entry.text)
        lines.append(lines_str.split("\n"))

    assert len(stations) == len(lines)
    assert len(stations) == len(areas)

    unique_stations, counts = np.unique(stations, 
                                        return_counts=True)
    for station in unique_stations[counts > 1]:
        i = stations.index(station)
        stations.pop(i+1)
        lines[i] = lines[i] + lines[i+1]
        areas.pop(i+1)

    return {
        "stations": stations,
        "lines": lines,
        "areas": areas
    }

In [ ]:
driver.get("https://en.wikipedia.org/wiki/Elizabeth_line#Route")

In [13]:
def find_stations_in_range(mat: np.array, start_ind: int, travel_time: float, stations_ind: list) -> None:
    '''
    Find stations within the travel time using recursion.
    '''
    if travel_time > 0:
        stations_ind.append(start_ind)
        for i in np.where(mat[start_ind] > 0)[0]:
            # Travel time from the current station to the neighbouring station.
            g_val: float = mat[start_ind, i]
            # Only check stations within the travel time and that have not been visited.
            if (g_val < travel_time) and (i not in stations_ind):
                find_stations_in_range(mat, i, travel_time - g_val, stations_ind)

In [3]:
with open("data.json", "r") as file:
    stations, lines, areas = json.load(file).values()

In [5]:
with open("nodes.json", "r") as file:
    nodes = json.load(file)

In [4]:
with open("times.npy", "rb") as file:
    mat = np.load(file)

In [ ]:
# Initialise mat.
# n = len(stations)
# mat = np.zeros((n, n), np.int8) 

In [32]:
route_url_dict = {
    "bakerloo": {
        "elc_haw": "https://www.google.com/maps/dir/Elephant+%26+Castle+Underground/Harrow+%26+Wealdstone,+The+Bridge,+High+St,+Harrow+HA3+5BP/@51.5541056,-0.2110221,11z/am=t/data=!4m14!4m13!1m5!1m1!1s0x487604a20531f45b:0x7da80f2ce5eb0ae0!2m2!1d-0.1003064!2d51.4946135!1m5!1m1!1s0x4876137941a06ff5:0xbb4f91708f98e221!2m2!1d-0.3343587!2d51.5917236!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwOS4wIKXMDSoASAFQAw%3D%3D"
    },
    "circle": {
        "ham_bla": "https://www.google.com/maps/dir/Hammersmith,+Underground+Ltd,+Hammersmith+Station,+Beadon+Road,+London/Blackfriars+station,+London/@51.5103531,-0.2330639,12z/am=t/data=!4m15!4m14!1m5!1m1!1s0x48760fb7def9c8bb:0xfb388ad277a5d7d4!2m2!1d-0.225012!2d51.4934334!1m5!1m1!1s0x487604adb1c8c98d:0xa906e9ebba170ebd!2m2!1d-0.1037435!2d51.5115979!3e3!5i1!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D",
        "pad_bla": "https://www.google.com/maps/dir/Paddington+Underground+Station,+Praed+Street,+London/Blackfriars+station,+London/@51.5081324,-0.2289845,12z/am=t/data=!4m14!4m13!1m5!1m1!1s0x48761ab2b05500a7:0x749d07ad72bbbe13!2m2!1d-0.1751595!2d51.5160334!1m5!1m1!1s0x487604adb1c8c98d:0xa906e9ebba170ebd!2m2!1d-0.1037435!2d51.5115979!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D"
    },
    "district": {
        "eab_upm": "https://www.google.com/maps/dir/''/Upminster,+Station+Road,+Upminster/@51.5338843,-0.189518,11z/am=t/data=!4m19!4m18!1m5!1m1!1s0x4876120a745961ad:0x9207ed0f81ac44ba!2m2!1d-0.3016676!2d51.5147193!1m5!1m1!1s0x47d8bbb30f7ea6a3:0x8916ea91832483ae!2m2!1d0.2506216!2d51.5589642!2m3!6e0!7e2!8j1749206400!3e3!5i1!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwMi4wIKXMDSoASAFQAw%3D%3D",
        "eac_edr": "https://www.google.com/maps/dir/Earl's+Court+Tube+Station,+Warwick+Road,+London/Edgware+Road,+Underground+Ltd,+Edgware+Road,+London/@51.5053269,-0.222952,13z/am=t/data=!4m14!4m13!1m5!1m1!1s0x48760f8c43d17cd3:0x7ceedb8e2a0f0a99!2m2!1d-0.1955577!2d51.4903065!1m5!1m1!1s0x48761ab6aa336037:0x6d85207552a2db32!2m2!1d-0.1701944!2d51.5202914!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D",
        "oly_eac": "https://www.google.com/maps/dir/Kensington+(Olympia),+Olympia+Way,+London/Earls+Court+Station,+Warwick+Road,+London/@51.493432,-0.221906,14z/am=t/data=!3m1!5s0x48761ab6aa58a1b9:0xccfbb15dd9570cf3!4m14!4m13!1m5!1m1!1s0x48760fea135f01e1:0xfda97938feb0aa15!2m2!1d-0.2099576!2d51.4979934!1m5!1m1!1s0x48760f0b16369355:0x6886f0c27f4aa00f!2m2!1d-0.1939564!2d51.4915677!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D",
        "tug_ric": "https://www.google.com/maps/dir/Turnham+Green+Station,+Turnham+Green+Terrace,+London/Richmond+underground,+Greater,+London/@51.4792264,-0.3193351,13z/am=t/data=!4m18!4m17!1m5!1m1!1s0x48760e3f137fc3e7:0x2a154bb6440aea35!2m2!1d-0.2545245!2d51.4951816!1m5!1m1!1s0x48760c377280f273:0x45540d669e54d16a!2m2!1d-0.3017462!2d51.4632798!2m3!6e0!7e2!8j1749206400!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D",
        "eac_wmb": "https://www.google.com/maps/dir/Earls+Court,+London/Wimbledon+Station,+London/@51.456709,-0.2845679,12z/am=t/data=!4m14!4m13!1m5!1m1!1s0x48760f8b593ccde9:0xbc7282217d8f10fc!2m2!1d-0.1958417!2d51.490331!1m5!1m1!1s0x487608b74564fba3:0x22621fbdfebb4dda!2m2!1d-0.2054832!2d51.4214125!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D",
    },
    "hammersmith": {
        "ham_bag": "https://www.google.com/maps/place/Hammersmith/@51.5519671,-0.2720873,11z/am=t/data=!4m23!1m16!4m15!1m6!1m2!1s0x48760fb49cf5881d:0xbaa0b2f34e5c12a8!2sHammersmith,+London!2m2!1d-0.223731!2d51.491187!1m6!1m2!1s0x47d8a590b42c329b:0x20d6d63bbad734a3!2sbarking!2m2!1d0.076127!2d51.5377369!5i1!3m5!1s0x48760fb7def9c8bb:0xfb388ad277a5d7d4!8m2!3d51.4934334!4d-0.225012!16s%2Fg%2F1hjgrnwff!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D"
    },
    "jubilee": {
        "stm_str": "https://www.google.com/maps/dir/Stanmore+Underground+Station,+Stanmore/Stratford+Underground+Station,+Greater,+London/@51.5583046,-0.1467687,11z/am=t/data=!3m1!4b1!4m14!4m13!1m5!1m1!1s0x4876143e2786d7cd:0xc30d509cbba8ee29!2m2!1d-0.3031205!2d51.6196885!1m5!1m1!1s0x48761d63d74e4981:0xe6b4025d0effd847!2m2!1d-0.003344!2d51.5411893!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwOS4wIKXMDSoASAFQAw%3D%3D"
    },
    "piccadilly": {
        "uxb_cks": "https://www.google.com/maps/dir/Uxbridge+Station,+Uxbridge/Cockfosters+Station,+Underground+Ltd,+Cockfosters+Road,+Barnet/@51.5613815,-0.4528026,11z/am=t/data=!3m1!4b1!4m14!4m13!1m5!1m1!1s0x48766e7a30465843:0x239ed54658f0b3ab!2m2!1d-0.478829!2d51.547665!1m5!1m1!1s0x487618509f0d73e1:0xd406d53ae08b37e5!2m2!1d-0.1497377!2d51.6516667!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D",
        "act_hiv": "https://www.google.com/maps/dir/Acton+Town,+Gunnersbury+Lane,+London/Heathrow+(LHR),+Terminal+4,+Nelson+Road,+Hounslow/@51.4821786,-0.4461374,12z/am=t/data=!4m14!4m13!1m5!1m1!1s0x48760e086bc508eb:0xa856a59bf4b41e3b!2m2!1d-0.2802019!2d51.502956!1m5!1m1!1s0x487673c790311477:0xc7c12ba131bb2c2d!2m2!1d-0.4473622!2d51.4596965!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D",
        "hrv": "https://www.google.com/maps/dir/Heathrow+Terminal+5,+Heathrow+Airport+(LHR),+Wallis+Road,+Longford,+Hounslow/Cockfosters+Station,+Underground+Ltd,+Cockfosters+Road,+Barnet/@51.5591049,-0.6219434,10z/am=t/data=!4m18!4m17!1m5!1m1!1s0x48767175ba8c845b:0xe983bc1ac0d4eefb!2m2!1d-0.4880069!2d51.4715021!1m5!1m1!1s0x487618509f0d73e1:0xd406d53ae08b37e5!2m2!1d-0.1497377!2d51.6516667!2m3!6e0!7e2!8j1749290400!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D"
    }
}

In [23]:
def update_times(mat):
    # Open journey details.
    for element in driver.find_elements(By.CSS_SELECTOR, 'button[aria-label="Toggle details"]'):
        if element.is_displayed():
            element.click()
    
    # Collect stations.
    journey = []
    for entry in driver.find_elements(By.CSS_SELECTOR, 'span[id^=transit_group] h2'):
        # Remove '.' from abbreviations in the station name.
        station: str = re.sub(r"\.", "", entry.text)
        # Remove 'Station' from the station name unless it refers to Battersea Power Station.
        station = re.sub(r"(?<!Battersea Power) Station", r"", station)
        if len(station) > 0:
            journey.append(station)

    # Collect timestamps.
    timestamps = []
    for entry in driver.find_elements(By.CSS_SELECTOR, 'span[id^=transit_group] div:has(~ h2), span[id^=transit_group] > div > div:first-child'):
        timestamp_str: str = re.sub(r"\u202f", "", entry.text)
        if len(timestamp_str) > 0:
            timestamp = datetime.strptime(timestamp_str, "%I:%M%p")
            timestamps.append(timestamp)

    # Calculate travel time between stations.
    times = []
    for i in range(len(timestamps) - 1):
        time: int = (timestamps[i+1] - timestamps[i]).seconds//60
        if time == 0:
            time = 1
        times.append(time)

    # Check extraction.
    assert len(journey) == (len(times) + 1)
    print(f'Number of stations: {len(journey)}')

    # Update time matrix.
    for i in range(len(times)):
        start_ind = stations.index(journey[i])
        end_ind = stations.index(journey[i+1])
        prev_time = mat[start_ind, end_ind]
        if prev_time == 0:
            mat[start_ind, end_ind] = times[i]
        else:
            mat[start_ind, end_ind] = 0.2 * times[i] + 0.8 * prev_time
        mat[end_ind, start_ind] = mat[start_ind, end_ind]
        print(f"{i}. Updated ({start_ind}, {end_ind}) from {prev_time} to {mat[start_ind, end_ind]}.")

    return {
        "times_mat": mat,
        "journey": journey
    }


In [33]:
# Requires user input to access and open travel details.
driver.get(route_url_dict["jubilee"]["stm_str"])

In [34]:
mat, journey = update_times(mat).values()

Number of stations: 27
0. Updated (215, 40) from 0.0 to 3.0.
1. Updated (40, 182) from 0.0 to 3.0.
2. Updated (182, 127) from 0.0 to 2.0.
3. Updated (127, 248) from 0.0 to 4.0.
4. Updated (248, 152) from 0.0 to 3.0.
5. Updated (152, 60) from 0.0 to 1.0.
6. Updated (60, 261) from 0.0 to 3.0.
7. Updated (261, 125) from 0.0 to 1.0.
8. Updated (125, 253) from 0.0 to 2.0.
9. Updated (253, 80) from 0.0 to 1.0.
10. Updated (80, 222) from 0.0 to 1.0.
11. Updated (222, 197) from 0.0 to 1.0.
12. Updated (197, 9) from 0.0 to 4.0.
13. Updated (9, 24) from 0.0 to 3.0.
14. Updated (24, 91) from 0.0 to 2.0.
15. Updated (91, 258) from 0.0 to 2.0.
16. Updated (258, 245) from 0.0 to 1.0.
17. Updated (245, 213) from 0.0 to 1.0.
18. Updated (213, 138) from 0.0 to 2.0.
19. Updated (138, 20) from 0.0 to 3.0.
20. Updated (20, 36) from 0.0 to 1.0.
21. Updated (36, 37) from 0.0 to 3.0.
22. Updated (37, 157) from 0.0 to 2.0.
23. Updated (157, 38) from 0.0 to 3.0.
24. Updated (38, 252) from 0.0 to 3.0.
25. Updat

In [35]:
nodes["jubilee"] = {
    "origin": journey[0],
    "stm_str": journey
}

In [36]:
with open("times.npy", "wb") as file:
    np.save(file, mat)

In [37]:
with open("nodes.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)

In [153]:
driver.quit()

In [ ]:
start = "South Kensington"
travel_time: float = 45
stations_ind = []
start_ind: int = stations.index(start)
find_stations_in_range(mat, start_ind, travel_time, stations_ind)


In [ ]:
set(areas[i] for i in stations_ind)

In [ ]:
# node_pos = np.zeros(len(stations))

In [10]:
piccadilly_nodes = np.zeros((len(stations), 3))

In [53]:
piccadilly_nodes[stations.index("Heathrow Terminal 5")]

array([14.,  9.,  2.])

In [26]:
for i in range(len(nodes["piccadilly"]["act_hiv"])):
    piccadilly_nodes[stations.index(nodes["piccadilly"]["act_hiv"][i])] = [14,i,0]

In [15]:
circle_nodes[stations.index("Bayswater")]

array([25.,  0.])

In [16]:
with open("circle.npy", "wb") as file:
    np.save(file, circle_nodes)

In [69]:
nodes = np.hstack((nodes, np.zeros((269,2))))

In [ ]:
nodes = np.zeros((len(stations), 2))

In [90]:
nodes[stations.index("Hammersmith")]

array([7., 0., 0., 0.])

In [91]:
nodes[stations.index("Edgware Road")]

array([10.,  0.,  5.,  0.])

In [92]:
nodes[stations.index("Paddington")]

array([10.,  0.,  4.,  0.])

In [83]:
i = 0
for station in journey:
    print(station)
    nodes[stations.index(station)] = [10,0,i,0]
    i -= 1

Earl's Court
West Brompton
Fulham Broadway
Parsons Green
Putney Bridge
East Putney
Southfields
Wimbledon Park
Wimbledon


In [51]:
with open("piccadilly.npy", "wb") as file:
    np.save(file, piccadilly_nodes)

In [54]:
driver.quit()

In [28]:
with open("nodes.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)